In [18]:
from sklearn.metrics import fbeta_score
y_true = [0, 1, 0, 0, 1, 1]
y_pred = [0, 0, 1, 1, 1, 1]
fbeta_score(y_true, y_pred, beta=1)



0.57142857142857151

In [10]:
fbeta_score(y_true, y_pred, beta=2)

0.625

In [12]:
fbeta_score(y_true, y_pred, beta=100)


0.66664444740701234

In [19]:
fbeta_score(y_true, y_pred, beta=1.2)

0.58653846153846145

In [ ]:
... 
0.23...
>>> fbeta_score(y_true, y_pred, average=None, beta=0.5)
... 
array([ 0.71...,  0.        ,  0.        ])